In [1]:
from proj_utils.fake_input_generator import generator
from proj_utils.input_interpreter import format_input

In [2]:

import matplotlib.pyplot as plt

from VRP.greedy_solutions import nearest_neighbour_solution,random_solution
from VRP.modify_solution import *
from VRP.verify_solution import valid_path, calculate_path_distance
from VRP.compare_solutions import num_different_edges


from proj_utils.path import print_path


import pandas as pd
import simulated_annealing.solver as SA
import taboo_search.solver as TS

from multi_agent.solver_model import MultiAgentSolverModel
from multi_agent.colaboration_types import ColaborationTypes
from solver_step import SolverStep
from multi_agent.solution_pool import solution_pool_simple
import pandas as pd
%matplotlib widget

In [3]:
customer_df = pd.read_excel('data/2_detail_table_customers.xls')
deposit_df = pd.read_excel('data/4_detail_table_depots.xls')
routes = customer_df['ROUTE_ID'].unique()
chosen_route = routes[2]
dists, demand, points = format_input(chosen_route,customer_df,deposit_df)
truck_volume_capacity = 30
truck_weight_capacity = 20000
truck_capacity = (truck_weight_capacity, truck_volume_capacity)

#dists, demand, points = generator(40)

In [4]:
customer_df = pd.read_excel('data/2_detail_table_customers.xls')
deposit_df = pd.read_excel('data/4_detail_table_depots.xls')
routes = customer_df['ROUTE_ID'].unique()
dists, demand, points = format_input(routes[0],customer_df,deposit_df)

In [5]:
routes

array([2946091, 2922001, 2939484, 2958047, 2970877, 2990001, 3005971,
       2604001, 3016355, 3027038, 3044702])

In [6]:
print(len(points))

79


In [7]:
eval_function = lambda x: calculate_path_distance(x, dists) if(valid_path(x,truck_capacity,demand)) else float('inf')

In [27]:
from q_learning import *

function_list = [
    lambda x:intra_route_swap(x,len(dists),dists),
    lambda x:inter_route_swap(x,len(dists),dists),
    lambda x:intra_route_shift(x,len(dists),dists),
    lambda x:inter_route_shift(x,len(dists),dists),
    lambda x:two_intra_route_shift(x,len(dists),dists),
    lambda x:two_intra_route_swap(x,len(dists),dists),
    lambda x:remove_smallest_route(x,len(dists),dists),
    lambda x:remove_random_route(x,len(dists),dists),
    lambda x:split_biggest_route(x,len(dists),dists),
    lambda x:split_random_route(x,len(dists),dists)
    ]

q = q_learning_obj(0,init_q_learning_table(len(function_list)),len(function_list), 0.9, 0.1)

#neighbour_function = lambda x: combined_rand_modification(x, len(points), dists)
neighbour_function = lambda x: q_learning_iteration(x,q,function_list,eval_function)

In [39]:
def genetic_algorithm_q_learning(solution, demandForCustomer, truckCapacityKg, truckCapacityVolume,q,function_list,eval_function):
    print('here')
    new_solution = q_learning_iteration(solution,q,function_list,eval_function)
    if(not valid_path(new_solution)):
        return solution

    return new_solution


In [28]:
solver_step_SA = SA.generic_solver_factory(
    neighbour_function,
    eval_function,
    )

In [29]:
taboo_list_size = 10
solver_step_TS = TS.taboo_search_solver_factory(
    neighbour_function,
    eval_function,
    taboo_list_size
    )

In [30]:
step_function_list = [solver_step_SA, solver_step_TS]
agent_labels = ['simulated annealing', 'taboo search']

In [31]:
init_state = random_solution(dists, demand, truck_capacity)
init_step = SolverStep(init_state,eval_function(init_state),100)

init_step_generator = lambda : SolverStep(random_solution(dists, demand, truck_capacity),eval_function(init_state),100)

In [40]:
solution_pool = solution_pool_simple(num_different_edges, radious=2)
sol_model = MultiAgentSolverModel(
    init_step_generator,
     step_function_list,
     chosen_route,20000,30,
     solution_pool,
     ColaborationTypes.FRIENDS,
     agent_labels = agent_labels,
     )

In [47]:
for _ in range(2):
    sol_model.step()

In [45]:
sol_states = sol_model.datacollector.get_model_vars_dataframe()
print(sol_states)

                                       TheGlobalBest  TheGlobalBestValue
0  [0, 1, 57, 17, 41, 66, 45, 68, 20, 60, 63, 48,...           30.882763
1  [0, 47, 66, 54, 24, 28, 74, 58, 52, 74, 37, 67...           19.277172
2  [0, 64, 67, 69, 56, 32, 55, 36, 37, 16, 7, 38,...           14.640011


In [46]:
agent_sol_states = sol_model.datacollector.get_agent_vars_dataframe()
print(agent_sol_states)

                      agent label  \
Step AgentID                        
1    0        simulated annealing   
     1               taboo search   
     2          genetic algorithm   
2    0        simulated annealing   
     1               taboo search   
     2          genetic algorithm   
3    0        simulated annealing   
     1               taboo search   
     2          genetic algorithm   

                                                      agentBest  \
Step AgentID                                                      
1    0        [0, 49, 72, 10, 35, 47, 51, 53, 9, 40, 43, 18,...   
     1        [0, 1, 57, 17, 41, 66, 45, 68, 20, 60, 63, 48,...   
     2                                                     None   
2    0        [0, 47, 66, 54, 24, 28, 74, 58, 52, 74, 37, 67...   
     1        [0, 37, 7, 2, 45, 1, 68, 20, 60, 73, 59, 48, 1...   
     2        [0, 35, 43, 27, 52, 4, 11, 24, 39, 62, 41, 45,...   
3    0        [0, 47, 66, 54, 24, 28, 74, 58, 52, 74, 37

In [17]:
agent_state = sol_model.datacollector.get_agent_vars_dataframe()
print(agent_state)
res = agent_state.unstack()
print(res)

                 Best                                           solution
Step AgentID                                                            
1    0            inf  [0, 1, 0, 68, 0, 10, 0, 93, 85, 0, 19, 0, 24, ...
     1            inf  [0, 1, 0, 68, 0, 10, 0, 93, 85, 0, 19, 0, 24, ...
     2        15.2872  [0, 96, 100, 76, 53, 51, 31, 86, 10, 18, 14, 7...
2    0        15.2872  [0, 96, 100, 76, 53, 51, 31, 86, 10, 18, 14, 7...
     1        15.2872  [0, 96, 100, 76, 53, 51, 31, 86, 10, 18, 14, 7...
...               ...                                                ...
49   1        10.3534  [0, 76, 100, 31, 78, 96, 23, 88, 39, 15, 26, 2...
     2        10.3534  [0, 76, 100, 31, 78, 96, 23, 88, 39, 15, 26, 2...
50   0        10.3534  [0, 76, 100, 31, 78, 96, 23, 88, 39, 15, 26, 2...
     1        10.3534  [0, 76, 100, 31, 78, 96, 23, 88, 39, 15, 26, 2...
     2        10.3534  [0, 76, 100, 31, 78, 96, 23, 88, 39, 15, 26, 2...

[150 rows x 2 columns]
            Best           

In [ ]:
for _ in range(100):
    sol_model.step()
    for sol in sol_model.solution_pool.pool:
        print(sol.get_best_sol())